In [25]:
import os
import re
from pathlib import Path
import pandas as pd

input_dir = Path(r"C:\Users\FahRe\Desktop\Apotheker-Agents\texts")
output_dir = Path(r"C:\Users\FahRe\Desktop\Apotheker-Agents\texts_cleaned")
output_dir.mkdir(exist_ok=True)

metadata_path = Path(r"C:\Users\FahRe\Downloads\Medication Guides.csv")
df_metadata = pd.read_csv(metadata_path)
df_metadata.set_index("Appl. No.", inplace=True)

SECTION_HEADERS = [
    "HIGHLIGHTS OF PRESCRIBING INFORMATION",
    "RECENT MAJOR CHANGES",
    "INDICATIONS AND USAGE",
    "DOSAGE AND ADMINISTRATION",
    "DOSAGE FORMS AND STRENGTHS",
    "CONTRAINDICATIONS",
    "WARNINGS AND PRECAUTIONS",
    "ADVERSE REACTIONS",
    "DRUG INTERACTIONS",
    "USE IN SPECIFIC POPULATIONS",
    "DRUG ABUSE AND DEPENDENCE",
    "OVERDOSAGE",
    "DESCRIPTION",
    "CLINICAL PHARMACOLOGY",
    "NONCLINICAL TOXICOLOGY",
    "HOW SUPPLIED/STORAGE AND HANDLING",
    "PATIENT COUNSELING INFORMATION",
]

FUZZY_REPLACEMENTS = {
    r"\bMAJ OR\b": "MAJOR",
    r"\bCOUN SELING\b": "COUNSELING",
    r"\bCONTRA INDICATIONS\b": "CONTRAINDICATIONS",
    r"\bNONCLIN ICAL\b": "NONCLINICAL",
    r"\bSUP PLIED\b": "SUPPLIED",
    r"\bADMIN IST RATION\b": "ADMINISTRATION",
    r"\bINFOR MATION\b": "INFORMATION",
    r"\bUSE IN SPECIFIC POPUL ATIONS\b": "USE IN SPECIFIC POPULATIONS",
}
section_patterns = [
    re.compile(rf"(?:^|\n)[#\-\s]*{re.escape(header)}[#\-\s]*\n?", re.IGNORECASE)
    for header in SECTION_HEADERS
]

def clean_and_tag_text(text: str) -> str:
    text = re.sub(r"Reference ID: \d+", "", text)
    text = re.sub(r"This label may not be the latest.*?drugsatfda", "", text, flags=re.DOTALL | re.IGNORECASE)
    text = re.sub(r"(\S)\n(\S)", r"\1 \2", text) 
    text = re.sub(r"\n{3,}", " ", text)  
    text = re.sub(r"[ \t]{3,}", " ", text)  #
    for broken, fixed in FUZZY_REPLACEMENTS.items():
        text = re.sub(broken, fixed, text, flags=re.IGNORECASE)
    for pattern, header in zip(section_patterns, SECTION_HEADERS):
        text = pattern.sub(f"\n\n### {header.upper()} ###\n\n", text)
    return text.strip()

results = []
for txt_file in input_dir.glob("*.txt"):
    appl_no = txt_file.stem
    try:
        with open(txt_file, "r", encoding="utf-8") as f:
            raw_text = f.read()

        if not raw_text.strip() or raw_text.startswith("[Error"):
            continue

        cleaned_text = clean_and_tag_text(raw_text)
        output_file = output_dir / f"{appl_no}.txt"
        with open(output_file, "w", encoding="utf-8") as f:
            f.write(cleaned_text)

        if int(appl_no) in df_metadata.index:
            row = df_metadata.loc[int(appl_no)]
            results.append({
                "application_no": appl_no,
                "cleaned_path": str(output_file),
                "drug_name": row["Drug Name"],
                "active_ingredient": row["Active Ingredient"],
                "form": row["Form; Route"],
                "company": row["Company"],
                "date": row["Date"],
                "url": row["URL"]
            })

    except Exception as e:
        print(f"Error processing {txt_file.name}: {e}")

df_results = pd.DataFrame(results)
